In [5]:
from datasets import Dataset

# Örnek veri seti
data = {
    "source": [
        "Bu, uzun bir haber metnidir ve önemli bilgileri içerir. Daha fazla detay için bu uzun metni özetlemek gereklidir.",
        "Başka bir haber örneği burada yer alıyor. Bu da detaylı bilgi içeriyor ve özetlenmesi isteniyor."
    ],
    "target": [
        "Bu haber önemli bilgileri içerir.",
        "Bu metin, detaylı bilgileri özetler."
    ]
}
dataset = Dataset.from_dict(data)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/llama-3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


ModuleNotFoundError: No module named 'transformers.utils'

In [1]:
# Otomatikleştirmek için gün belirleme

from datetime import datetime, timedelta

# Bugünün tarihi
today = datetime.today()
# Dünün tarihi
yesterday = today - timedelta(days=1)
# Tarih formatları ekonomim.com için
formatted_today = today.strftime("%d-%m-%Y")
formatted_yesterday = yesterday.strftime("%d-%m-%Y")

# Pazartesi gününü anlamak için kod
def check_monday():
    if today.weekday() == 0:
        print("Bugün Pazartesi.")
        return formatted_today, formatted_yesterday
    else:
        print("Bugün Pazartesi değil.")
        return formatted_today, formatted_yesterday

# Fonksiyonu çağır
today  = check_monday()
print("Bugünün tarihi:", formatted_today)
print("Dünün tarihi:", formatted_yesterday)


Bugün Pazartesi değil.
Bugünün tarihi: 14-11-2024
Dünün tarihi: 13-11-2024


In [3]:
# Ekonomimde dünün haberlerini açıyor

# Required Libraries
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
warnings.filterwarnings('ignore')

# !!!!Download Location!!!!
download_dir = r"C:\Users\oguzo\Desktop\Gundem-Automation"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")
# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")
# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://www.ekonomim.com/son-dakika-haberleri')

# Dünün tarihini hesaplayın
yesterday = datetime.today() - timedelta(days=1)
formatted_yesterday = yesterday.strftime("%d-%m-%Y")

# Sayfanın tamamen yüklenmesini bekleyin (alternatif olarak explicit waits kullanabilirsiniz)
time.sleep(2)

try:
    # XPath ile input alanını bulun
    input_field = WebDriverWait(browser, 30).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/main/div[2]/div[1]/form/input"))
    )
    
    # Mevcut değeri temizleyin
    input_field.clear()
    
    # Yeni değeri girin
    input_field.send_keys(formatted_yesterday)

    # XPath ile düğmeyi bulun ve tıklayın
    search_button = WebDriverWait(browser, 30).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/main/div[2]/div[1]/form/button"))
    )
    search_button.click()

except (TimeoutException, StaleElementReferenceException) as e:
    print(f"An error occurred: {e}")
finally:
    time.sleep(5)
    browser.quit()

In [ ]:
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get(export GROQ_API_KEY="aa"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Build a turkish news resume automation in 2 sentence for not loosing time."
        },
        {
            "role": "user",
            "content": "Resume this in 2 sentence in turkish",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [ ]:
import groq
import json
from jinja2 import Template
import datetime

# Define the Groq query
query = """
queries news {
  stories {
    title
    created_at
    author {
      name
    }
    category {
      name
    }
    summary
    content
  }
}
"""

# Define the API endpoint and API key
api_endpoint = "https://api.groqi.com/v1/graphql"
api_key = "aa"

# Define the template
template = Template('''
Resume for {{ title }}
================================

**Title:** {{ title }}
**Date:** {{ date }}
**Author:** {{ author }}
**Category:** {{ category }}

**Summary:**

{{ summary }}

**Content:**

{{ content | strip_html | truncate(500) }}


**End of Story**
''')

# Define the scraping function
def scrape_news(api_endpoint, api_key, query):
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    data = json.dumps({"query": query})
    response = requests.post(api_endpoint, headers=headers, data=data)

    if response.status_code == 200:
        data = response.json()
        articles = data["data"]["stories"]
        for article in articles:
            yield {
                "title": article["title"],
                "date": article["created_at"],
                "author": article["author"]["name"],
                "category": article["category"]["name"],
                "summary": article["summary"],
                "content": article["content"]
            }
    else:
        print(f"Failed to retrieve data: {response.status_code}")

# Scrape the news articles
for article in scrape_news(api_endpoint, api_key, query):
    resume = template.render(
        title=article['title'],
        date=article['date'],
        author=article['author'],
        category=article['category'],
        summary=article['summary'],
        content=article['content']
    )

    # Write the resume to a file
    with open(f"{article['title']}.txt", 'w') as f:
        f.write(resume)

    print(f"Generated resume for {article['title']}")

TemplateAssertionError: No filter named 'strip_html'.

In [7]:
# Code For Published Data Timeline

# Required Libraries
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
warnings.filterwarnings('ignore')

# !!!!Download Location!!!!
download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")
# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")
# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')


first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)


second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


In [ ]:
# BeautifulSoup ile verileri çekme
def get_data(soup):
    # class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
    class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

    # class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
    class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

    # İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
    max_length = max(len(class_1_data), len(class_2_data))
    class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
    class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

    return pd.DataFrame({
        'Class_1_Data': class_1_data,
        'Class_2_Data': class_2_data
    })

all_data = pd.DataFrame()
# 1'den 10'a kadar döngü ile veriyi çekiyoruz
for i in range(2, 11):  # li[1] ile li[10] arasındaki verileri çeker
    # Second click için XPath oluşturma
    second_xpath = f"/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[{i}]/div[2]"
    
    try:
        # Second click (ilk tıklama)
        second_click = WebDriverWait(browser, 30).until(
            EC.element_to_be_clickable((By.XPATH, second_xpath))
        )
        browser.execute_script("arguments[0].click();", second_click)

        # Verileri çekmek için BeautifulSoup ile sayfa kaynağını al
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        df = get_data(soup)

        # Yeni çekilen veriyi all_data DataFrame'ine ekle
        all_data = pd.concat([all_data, df], ignore_index=True)

        # Zero click (bu işlem verileri aldıktan sonra yapılacak)
        time.sleep(5)
        zero_click = WebDriverWait(browser, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
        )
        browser.execute_script("arguments[0].click();", zero_click)

        # Aynı öğeyi tekrar kapatma (second click ile aynı element)
        browser.execute_script("arguments[0].click();", second_click)
    
    except TimeoutException:
        print(f"Element {i} bulunamadı, devam ediliyor.")
        continue

# Tüm verileri yazdır
print(df, all_data)


In [ ]:
/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]
/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]
/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[2]/div[2]


/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[10]/div[2]
,/html/body/div[1]/div/div[4]/div/div[2]/button[2]

In [2]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                         Class_1_Data Class_2_Data
0                                   Textiles (0 HS92)     (0 HS92)
1                                      Silk (50 HS92)    (50 HS92)
2                                      Wool (51 HS92)    (51 HS92)
3                                    Cotton (52 HS92)    (52 HS92)
4            Other vegetable textile fibres (53 HS92)    (53 HS92)
5                        Man-made filaments (54 HS92)    (54 HS92)
6                    Man-made staple fibres (55 HS92)    (55 HS92)
7               Wadding, felt and nonwovens (56 HS92)    (56 HS92)
8                                   Carpets (57 HS92)    (57 HS92)
9                     Special woven fabrics (58 HS92)    (58 HS92)
10  Impregnated, coated or laminated textile fabri...    (59 HS92)
11                          Knitted fabrics (60 HS92)    (60 HS92)
12                            Apparel, knit (61 HS92)    (61 HS92)
13                        Apparel, not knit (62 HS92)    (62 H

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [3]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[2]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df1 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df1)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[2]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                         Class_1_Data Class_2_Data
0                                   Textiles (0 HS92)     (0 HS92)
1                                Agriculture (1 HS92)     (1 HS92)
2                              Live animals (01 HS92)    (01 HS92)
3                                      Meat (02 HS92)    (02 HS92)
4                                      Fish (03 HS92)    (03 HS92)
5                            Diary products (04 HS92)    (04 HS92)
6                           Animal products (05 HS92)    (05 HS92)
7                                    Plants (06 HS92)    (06 HS92)
8                                Vegetables (07 HS92)    (07 HS92)
9                           Fruits and nuts (08 HS92)    (08 HS92)
10                   Coffee, tea and spices (09 HS92)    (09 HS92)
11                                  Cereals (10 HS92)    (10 HS92)
12               Flours, starches and malts (11 HS92)    (11 HS92)
13          Oil seeds and oleaginous fruits (12 HS92)    (12 H

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [4]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[3]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df2 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df2)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[3]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                         Class_1_Data Class_2_Data
0                                   Textiles (0 HS92)     (0 HS92)
1                                Agriculture (1 HS92)     (1 HS92)
2                                      Stone (2 HS92)     (2 HS92)
3   Articles of stone, plaster, cement, etc. (68 H...    (68 HS92)
4                          Ceramic products (69 HS92)    (69 HS92)
5                       Glass and glassware (70 HS92)    (70 HS92)
6                Precious metals and stones (71 HS92)    (71 HS92)
7                                   Minerals (3 HS92)     (3 HS92)
8                                     Metals (4 HS92)     (4 HS92)
9                                  Chemicals (5 HS92)     (5 HS92)
10                                  Vehicles (6 HS92)     (6 HS92)
11                                 Machinery (7 HS92)     (7 HS92)
12                               Electronics (8 HS92)     (8 HS92)
13                                     Other (9 HS92)     (9 H

In [5]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[4]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df3 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df3)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[4]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                   Class_1_Data Class_2_Data
0                             Textiles (0 HS92)     (0 HS92)
1                          Agriculture (1 HS92)     (1 HS92)
2                                Stone (2 HS92)     (2 HS92)
3                             Minerals (3 HS92)     (3 HS92)
4   Salt, sulphur, lime, cement, etc. (25 HS92)    (25 HS92)
5                  Ores, slag and ash (26 HS92)    (26 HS92)
6       Mineral fuels, oils and waxes (27 HS92)    (27 HS92)
7                               Metals (4 HS92)     (4 HS92)
8                            Chemicals (5 HS92)     (5 HS92)
9                             Vehicles (6 HS92)     (6 HS92)
10                           Machinery (7 HS92)     (7 HS92)
11                         Electronics (8 HS92)     (8 HS92)
12                               Other (9 HS92)     (9 HS92)
13                                     Services          NaN


In [6]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[5]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df4 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df4)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[5]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                      Class_1_Data Class_2_Data
0                                Textiles (0 HS92)     (0 HS92)
1                             Agriculture (1 HS92)     (1 HS92)
2                                   Stone (2 HS92)     (2 HS92)
3                                Minerals (3 HS92)     (3 HS92)
4                                  Metals (4 HS92)     (4 HS92)
5                         Iron and steel (72 HS92)    (72 HS92)
6              Articles of iron or steel (73 HS92)    (73 HS92)
7                                 Copper (74 HS92)    (74 HS92)
8                                 Nickel (75 HS92)    (75 HS92)
9                              Aluminium (76 HS92)    (76 HS92)
10                                  Lead (78 HS92)    (78 HS92)
11                                  Zinc (79 HS92)    (79 HS92)
12                                   Tin (80 HS92)    (80 HS92)
13                     Other base metals (81 HS92)    (81 HS92)
14             Metal tools and tableware

In [7]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[6]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df5 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df5)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[6]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                         Class_1_Data Class_2_Data
0                                   Textiles (0 HS92)     (0 HS92)
1                                Agriculture (1 HS92)     (1 HS92)
2                                      Stone (2 HS92)     (2 HS92)
3                                   Minerals (3 HS92)     (3 HS92)
4                                     Metals (4 HS92)     (4 HS92)
5                                  Chemicals (5 HS92)     (5 HS92)
6                       Inorganic chemicals (28 HS92)    (28 HS92)
7                         Organic chemicals (29 HS92)    (29 HS92)
8                   Pharmaceutical products (30 HS92)    (30 HS92)
9                               Fertilisers (31 HS92)    (31 HS92)
10                 Dyes, paints, inks, etc. (32 HS92)    (32 HS92)
11                           Essential oils (33 HS92)    (33 HS92)
12                 Soaps, waxes, and paints (34 HS92)    (34 HS92)
13  Albuminoidals; modified starches; glues; enzym...    (35 H

In [8]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[7]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df6 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df6)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[7]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


            Class_1_Data Class_2_Data
0      Textiles (0 HS92)     (0 HS92)
1   Agriculture (1 HS92)     (1 HS92)
2         Stone (2 HS92)     (2 HS92)
3      Minerals (3 HS92)     (3 HS92)
4        Metals (4 HS92)     (4 HS92)
5     Chemicals (5 HS92)     (5 HS92)
6      Vehicles (6 HS92)     (6 HS92)
7       Trains (86 HS92)    (86 HS92)
8     Vehicles (87 HS92)    (87 HS92)
9     Aircraft (88 HS92)    (88 HS92)
10       Ships (89 HS92)    (89 HS92)
11    Machinery (7 HS92)     (7 HS92)
12  Electronics (8 HS92)     (8 HS92)
13        Other (9 HS92)     (9 HS92)
14              Services          NaN


In [9]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[8]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df7 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df7)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[8]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                      Class_1_Data Class_2_Data
0                                Textiles (0 HS92)     (0 HS92)
1                             Agriculture (1 HS92)     (1 HS92)
2                                   Stone (2 HS92)     (2 HS92)
3                                Minerals (3 HS92)     (3 HS92)
4                                  Metals (4 HS92)     (4 HS92)
5                               Chemicals (5 HS92)     (5 HS92)
6                                Vehicles (6 HS92)     (6 HS92)
7                               Machinery (7 HS92)     (7 HS92)
8                   Industrial Machinery (84 HS92)    (84 HS92)
9   Apparatuses (optical, medical, etc.) (90 HS92)    (90 HS92)
10                                Clocks (91 HS92)    (91 HS92)
11                   Musical instruments (92 HS92)    (92 HS92)
12                   Arms and ammunition (93 HS92)    (93 HS92)
13                                  Toys (95 HS92)    (95 HS92)
14   Miscellaneous manufactured articles

In [10]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[9]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df8 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df8)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[9]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                    Class_1_Data Class_2_Data
0                              Textiles (0 HS92)     (0 HS92)
1                           Agriculture (1 HS92)     (1 HS92)
2                                 Stone (2 HS92)     (2 HS92)
3                              Minerals (3 HS92)     (3 HS92)
4                                Metals (4 HS92)     (4 HS92)
5                             Chemicals (5 HS92)     (5 HS92)
6                              Vehicles (6 HS92)     (6 HS92)
7                             Machinery (7 HS92)     (7 HS92)
8                           Electronics (8 HS92)     (8 HS92)
9   Electrical machinery and equipment (85 HS92)    (85 HS92)
10                                Other (9 HS92)     (9 HS92)
11                                      Services          NaN


In [11]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np
warnings.filterwarnings('ignore')

download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
time.sleep(30)
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)
# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[10]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df9 = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

print(df9)

# Zero click
time.sleep(5)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[10]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


            Class_1_Data Class_2_Data
0      Textiles (0 HS92)     (0 HS92)
1   Agriculture (1 HS92)     (1 HS92)
2         Stone (2 HS92)     (2 HS92)
3      Minerals (3 HS92)     (3 HS92)
4        Metals (4 HS92)     (4 HS92)
5     Chemicals (5 HS92)     (5 HS92)
6      Vehicles (6 HS92)     (6 HS92)
7     Machinery (7 HS92)     (7 HS92)
8   Electronics (8 HS92)     (8 HS92)
9         Other (9 HS92)     (9 HS92)
10       Other (99 HS92)    (99 HS92)
11              Services          NaN


In [12]:
import pandas as pd

# Tüm veri çerçevelerini bir listeye koy
dataframes = [df, df1, df2, df3, df4, df5, df6, df7, df8, df9]

# Hepsini birleştir
combined_df = pd.concat(dataframes, ignore_index=True)
# Excel dosyasına yaz
combined_df.to_excel("combined_atlas_data.xlsx", index=False)


In [23]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np

warnings.filterwarnings('ignore')

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim")
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)

# BeautifulSoup ile verileri çekme
def get_data(soup):
    # class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
    class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

    # class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
    class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

    # İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
    max_length = max(len(class_1_data), len(class_2_data))
    class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
    class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

    return pd.DataFrame({
        'Class_1_Data': class_1_data,
        'Class_2_Data': class_2_data
    })

# Veri toplama döngüsü
all_data = pd.DataFrame()  # Tüm verilerin toplandığı DataFrame

# 1'den 10'a kadar döngü ile veriyi çekiyoruz
for i in range(1, 11):  # li[1] ile li[10] arasındaki verileri çeker
    # Second click için XPath oluşturma
    second_xpath = f"/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[{i}]/div[2]"
    
    try:
        # Second click (ilk tıklama)
        second_click = WebDriverWait(browser, 30).until(
            EC.element_to_be_clickable((By.XPATH, second_xpath))
        )
        browser.execute_script("arguments[0].click();", second_click)

        # Verileri çekmek için BeautifulSoup ile sayfa kaynağını al
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        df = get_data(soup)

        # Yeni çekilen veriyi all_data DataFrame'ine ekle
        all_data = pd.concat([all_data, df], ignore_index=True)

        # Zero click (bu işlem verileri aldıktan sonra yapılacak)
        time.sleep(5)
        zero_click = WebDriverWait(browser, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
        )
        browser.execute_script("arguments[0].click();", zero_click)

        # Aynı öğeyi tekrar kapatma (second click ile aynı element)
        browser.execute_script("arguments[0].click();", second_click)
    
    except TimeoutException:
        print(f"Element {i} bulunamadı, devam ediliyor.")
        continue

# Tüm verileri yazdır
print(all_data)

# Tarayıcıyı kapatma
browser.quit()


Element 2 bulunamadı, devam ediliyor.
Element 3 bulunamadı, devam ediliyor.
Element 4 bulunamadı, devam ediliyor.


KeyboardInterrupt: 

In [20]:
df

,Class_1_Data,Class_2_Data
0,Textiles (0 HS92),(0 HS92)
1,Silk (50 HS92),(50 HS92)
2,Wool (51 HS92),(51 HS92)
3,Cotton (52 HS92),(52 HS92)
4,Other vegetable textile fibres (53 HS92),(53 HS92)
5,Man-made filaments (54 HS92),(54 HS92)
6,Man-made staple fibres (55 HS92),(55 HS92)
7,"Wadding, felt and nonwovens (56 HS92)",(56 HS92)
8,Carpets (57 HS92),(57 HS92)
9,Special woven fabrics (58 HS92),(58 HS92)
